## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [10]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier                           # GBM
from sklearn.ensemble import RandomForestClassifier# 隨機森林
from sklearn.model_selection import GridSearchCV



# 1. get data
digits = datasets.load_digits()

In [11]:
#2. split data
x_train,x_test,y_train,y_test = train_test_split(digits.data,digits.target,test_size=0.3,random_state=4)

In [12]:
# 3.1 建立Model
clf = GradientBoostingClassifier (loss = 'deviance',learning_rate=0.1, n_estimators=100)
#clf = RandomForestClassifier()
# 3.2 訓練Model
clf.fit(x_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [13]:
# 4. 預測
y_predict = clf.predict(x_test)

In [14]:
# 5. 評分
metrics.accuracy_score(y_predict,y_test)

0.9648148148148148

In [21]:
# 6.1 
n_estimators = [50, 100, 150]
learning_rate = [0.1, 0.2,0.3]
param_grid = dict(n_estimators=n_estimators, learning_rate = learning_rate)

# 6.2.1
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)   #Model
# 6.2.2
grid_result = grid_search.fit(x_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   21.6s finished


In [22]:
grid_result.best_params_

{'learning_rate': 0.1, 'n_estimators': 150}

In [27]:
clf = GradientBoostingClassifier (loss = 'deviance',learning_rate=0.1, n_estimators=150)
clf.fit(x_train,y_train)
y_predict = clf.predict(x_test)
metrics.accuracy_score(y_predict,y_test)

0.9648148148148148